In [1]:
import datajoint as dj
import numpy as np
import time



In [2]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

schema = dj.schema('microns_pinky')
pinky = dj.create_virtual_module('pinky', 'microns_pinky')

Connecting celiib@10.28.0.34:3306


In [3]:
pinky.ExcitatoryLeftoverMeshes & pinky.CurrentSegmentation & 'decimation_ratio=0.35' & pinky.LeftoverCoarseLabelFinal.proj() & "n_triangles>0"


segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,n_vertices number of vertices in this mesh pieces that were filtered away,n_triangles number of triangles in this mesh pieces that were filtered away,"vertices x,y,z coordinates of vertices",triangles triangles (triplets of vertices),recovered_perc number of faces of this recovered mesh / number of faces in filtered pymeshfix mesh
3,648518346341371119,0.35,287,489,=BLOB=,=BLOB=,0.00109
3,648518346349386137,0.35,66,100,=BLOB=,=BLOB=,0.00178
3,648518346349470171,0.35,31607,62878,=BLOB=,=BLOB=,0.08902
3,648518346349471156,0.35,2220,4339,=BLOB=,=BLOB=,0.00842
3,648518346349471500,0.35,128,235,=BLOB=,=BLOB=,0.00051
3,648518346349471562,0.35,3969,7687,=BLOB=,=BLOB=,0.00857
3,648518346349471565,0.35,335,543,=BLOB=,=BLOB=,0.00036
3,648518346349471910,0.35,1034,1988,=BLOB=,=BLOB=,0.00151
3,648518346349472574,0.35,649,1208,=BLOB=,=BLOB=,0.00115
3,648518346349472601,0.35,106,199,=BLOB=,=BLOB=,0.00019


In [4]:
#new imports for the revised compartments components code
import trimesh
from trimesh.graph import connected_components

In [5]:
def celii_split(mesh,
          only_watertight=False,
          adjacency=None,
          engine=None):
    """
    Split a mesh into multiple meshes from face connectivity.
    If only_watertight is true, it will only return watertight meshes
    and will attempt single triangle/quad repairs.
    Parameters
    ----------
    mesh: Trimesh
    only_watertight: if True, only return watertight components
    adjacency: (n,2) list of face adjacency to override using the plain
               adjacency calculated automatically.
    engine: str, which engine to use. ('networkx', 'scipy', or 'graphtool')
    Returns
    ----------
    meshes: list of Trimesh objects
    """
    
    
    if adjacency is None:
        adjacency = mesh.face_adjacency

    # if only watertight the shortest thing we can split has 3 triangles
    if only_watertight:
        min_len = 3
    else:
        min_len = 1

    components = connected_components(edges=adjacency,
                                      nodes=np.arange(len(mesh.faces)),
                                      min_len=min_len,
                                      engine=engine)
    return components

In [6]:
def get_vertex_indices(split_face_indices,total_mesh):
    """
    Will generate the vertex indices from the original face
    based on the faces indices passed along with the original mesh
    (can accept just one face indices arrray or list of arrays)
    
    Pseudocode: 
    1) Get the original faces array
    2) Get a list of all of the vertices indexes from the face indices
    3) get a unique list and return it
    """
    
    vertex_indices = []

    original_face_list = np.array(list(total_mesh.faces))

    for face_indices in split_face_indices:
        #print(face_indices)
        #get the corresponding vertices indexes
        #print(original_face_list[face_indices].ravel())
        unique_vert_indexes = np.unique(original_face_list[face_indices].ravel())
        #print(unique_vert_indexes)
        vertex_indices.append(unique_vert_indexes)

    return vertex_indices
        
    

In [7]:
def make_datajoint_keys(key,label_name,vertex_indices,face_indexes_real):
    #make sure they both have the same length
    #print(f"len(vertex_indices) = " + str(len(vertex_indices)))
    #print(f"len(face_indexes_real) = " + str(len(face_indexes_real)))
    
    if len(face_indexes_real) != len(vertex_indices):
        raise Exception("Length of vertex indices and face indices lists do not match in make datajoint keys")
    
    #print(f"face_indexes_real = {face_indexes_real}")
    #print(f"vertex_indices = {vertex_indices}")
    key_list = []
    for j in range(0,len(face_indexes_real)):
        vertex_list = vertex_indices[j]
        face_list = face_indexes_real[j]
        
        current_key = dict(key,
                           compartment_type = label_name,
                           component_index=j,
                           n_vertex_indices = len(vertex_list),
                           n_triangle_indices = len(face_list),
                           vertex_indices = vertex_list,
                           triangle_indices = face_list
                          )
        key_list.append(current_key)
    return key_list
        

In [8]:
@schema
class LeftoverCompartmentFinal(dj.Computed):
    definition = """
    -> pinky.ExcitatoryLeftoverMeshes
    ---
    """

    class LeftoverComponentFinal(dj.Part):
        definition = """
        -> LeftoverCompartmentFinal
        compartment_type   : varchar(16)        # Basal, Apical, spine head, etc.
        component_index    : smallint unsigned  # Which sub-compartment of a certain label this is.
        ---
        n_vertex_indices   : bigint
        n_triangle_indices : bigint
        vertex_indices     : longblob           # preserved indices of each vertex of this sub-compartment
        triangle_indices   : longblob           # preserved indices of each triangle of this sub-compartment
        """
    
    key_source = pinky.ExcitatoryLeftoverMeshes & pinky.CurrentSegmentation & 'decimation_ratio=0.35' & pinky.LeftoverCoarseLabelFinal.proj() & "n_triangles>0"

    
    def make(self, key):
        print(f"\n\n -------- Working on {key['segment_id']}----------")
        
        start = time.time()
        
        #get the full mesh
        mesh = (pinky.ExcitatoryLeftoverMeshes & key).fetch1()
        verts = mesh["vertices"]
        faces = mesh["triangles"]
        
        #get the labels
        labels = (pinky.LeftoverCoarseLabelFinal & key).fetch1()
        verts_labels = labels["vertices"]
        faces_labels = labels["triangles"]
        
#         #check to see if thre are currently no triangles for the labels, and if so write dummy value
              
#         if len(np.unique(labels['triangles'])) == 1:
#             #print("heyo")
#             self.insert1(key)
#             label_name = (pinky.LabelKey & dict(numeric=np.unique(labels['triangles'])[0])).fetch1('description')
#             vertex_indices = np.arange(len(labels['vertices']), dtype=np.uint32)
#             triangle_indices = np.arange(len(labels['triangles']), dtype=np.uint32)
#             new_dict= dict(key,
#                                                 compartment_type=label_name,
#                                                 component_index=0,
#                                                 n_vertex_indices=len(vertex_indices),
#                                                 n_triangle_indices=len(triangle_indices),
#                                                 vertex_indices=vertex_indices,
#                                                 triangle_indices=triangle_indices)
            
#             LeftoverCompartmentFinal.LeftoverComponentFinal().insert1(dict(key,
#                                                 compartment_type=label_name,
#                                                 component_index=0,
#                                                 n_vertex_indices=len(vertex_indices),
#                                                 n_triangle_indices=len(triangle_indices),
#                                                 vertex_indices=vertex_indices,
#                                                 triangle_indices=triangle_indices))
#             return
        
        numeric,description = pinky.LabelKey.fetch("numeric","description")
        labels_dict = dict(zip(numeric,description))
        
        #get the number of unique labels:
        unique_labels = np.unique(verts_labels)
        print("unique_labels = " + str(unique_labels))
        
        #create the Trimesh of the whole mesh
        total_mesh = trimesh.Trimesh(verts,faces,process=False)
        
        #iterate through all of the different labels
        total_keys = []
        for label in unique_labels:
            print("Working on label " + str(label))
            face_indexes = np.where(faces_labels==label)[0]
            #create a submesh
            new_mesh = total_mesh.submesh([face_indexes],only_watertight=False,append=False)[0]#.split()
            split_face_indices = celii_split(new_mesh)

            #print(f"faces_indexes = {face_indexes}")
            #print(f"split_face_indices = {np.array(split_face_indices)}")

            face_indexes_real = []
            for sp in split_face_indices:
                face_indexes_real.append(face_indexes[sp])

            #get the real indices 
            #face_indexes_real = face_indexes[np.array(split_face_indices)]

            vertex_indices = get_vertex_indices(face_indexes_real,total_mesh)

            dj_keys = make_datajoint_keys(key,labels_dict[label],vertex_indices,face_indexes_real)

            total_keys = total_keys + dj_keys

        
        self.insert1(key)
        print("\nTotal number of keys inserting = " + str(len(total_keys)))
        LeftoverCompartmentFinal.LeftoverComponentFinal().insert(total_keys)

        print(key['segment_id'], "finished separating components:", time.time() - start)

In [9]:
#(schema.jobs & "table_name='__leftover_compartment_final'").delete()

In [10]:
#pinky.LeftoverCompartmentFinal.drop()

In [11]:
start_time = time.time()
LeftoverCompartmentFinal.populate(reserve_jobs=True)
print(f"Total time = {time.time() - start_time}")



 -------- Working on 648518346341371119----------
unique_labels = [2]
Working on label 2

Total number of keys inserting = 3
648518346341371119 finished separating components: 0.07037043571472168


 -------- Working on 648518346349386137----------
unique_labels = [5]
Working on label 5

Total number of keys inserting = 1
648518346349386137 finished separating components: 0.1003427505493164


 -------- Working on 648518346349470171----------
unique_labels = [3 6]
Working on label 3
Working on label 6

Total number of keys inserting = 24
648518346349470171 finished separating components: 0.6968390941619873


 -------- Working on 648518346349471156----------
unique_labels = [3 6]
Working on label 3
Working on label 6

Total number of keys inserting = 5
648518346349471156 finished separating components: 0.14338898658752441


 -------- Working on 648518346349471500----------
unique_labels = [3]
Working on label 3

Total number of keys inserting = 2
648518346349471500 finished separating c

In [12]:
pinky.LeftoverCompartmentFinal.LeftoverComponentFinal()

AttributeError: module 'pinky' has no attribute 'LeftoverCompartmentFinal'

# check that all neurons have components

In [ ]:
#(schema.jobs & "table_name='__compartment_final'").delete()

In [ ]:
# #check that there are all components in there
# pinky.CompartmentFinal.ComponentFinal()

In [ ]:
# segment_ids = pinky.CompartmentFinal.ComponentFinal.fetch("segment_id")
# segment_ids

In [ ]:
# from collections import Counter
# segment_id_counters = Counter(segment_ids)
# #segment_id_counters

In [ ]:
# for i in segment_id_counters.values():
#     if i == 0:
#         print("0 value")

In [ ]:
# print(len(segment_id_counters.keys()))
# print(np.sum(np.where(segment_id_counters.values() > 0)))

In [ ]:
"""
Conclusion: There were all segments present and no segments that had 0 parts
So it is ready for spine computation because all have passed


"""